# Getting Started - chatsnack Snacking Guide

## Setup

### Got snack?
Install the `chatsnack` package from PyPI.

In [ ]:
!pip install chatsnack

### Got API key?
If you haven't already, add your OpenAI API key to a .env file. This cell will check if you have .env and create a new one for you if needed. We use env.template.cataclysm as an example.

In [ ]:
# Got the .env file? It is where you put your OpenAI API key, so you'll need it.
import os
# we also want to ensure we have a .env file that contains our API keys, so warn if we don't have that file present in this directory
if not os.path.exists(".env"):
    print("WARNING: No .env file found in this directory. Please create one with the API Key contents mentioned in the README.md file.")

# we don't need logs during the demo
from loguru import logger
logger.disable("chatsnack")


## Have a Quick Snack
The simplest way to get started is via built-in snack packs. Each pack is a singleton ready to mingleton.

In [ ]:
from chatsnack.packs import ChatsnackHelp
ChatsnackHelp.ask("What is your primary directive?")

`ask()` returns response as a string, but *doesn't change the chat object*.

When you want to keep the conversation continuing, use `chat()` instead-- it returns a new `Chat` object you can use:

In [ ]:
mychat = ChatsnackHelp.chat("What is chatsnack?")
print(mychat.response)

In [ ]:
# let's continue the chat, but we have new stipulations and will speak for the AI to make it think it is already compliant
mychat.user("Respond in only six word sentences from now on.")
mychat.asst("I promise I will do so.")
mychat.user("How should I spend my day?")
mychat.ask()

In [ ]:
print(f"ask() doesn't store the result in the chat object:\n\t{mychat.last}\n")
print("Ask again: " + mychat.ask() + "\n")
print(f"See? .last is still the same after ask():\n\t{mychat.last}\n\n")

# but if we want a full conversation, use chat() instead
newchat = mychat.chat()
print(f"chat() gives us a new Chat object with response included for easy continuation:\n\t{newchat.last}\n")

newchat = (
    newchat
    .user("Do you have a name and what does it mean?")
    .chat()
)
print(f"Ooo-- did you notice that? We used .user().chat() and chained those together:\n\t{newchat.last}\n")

Chaining like that works pretty well with `.chat()` and the other messages.

You can also shortcut and pass a string to `.chat()` and it will prepare a user message for you.

In [ ]:
print(newchat.chat("What about my name Bob?").response)

Want to have an interactive conversation loop? Here's example code for that:

In [ ]:
from chatsnack.packs import Jolly
yourchat = Jolly
while (user_input := input("Chat with the bot: ")):
    print(f"USER: {user_input}")
    yourchat = yourchat.chat(user_input)
    print(f"THEM: {yourchat.last}")

### Cooking Temperature
You can change OpenAI parameters in each chat, supporting the `gpt-4` and `gpt-3.5-turbo` families. Right now the default is `gpt-3.5-turbo`, but when `gpt-4` is released widely, it will become the default.

In [ ]:
from chatsnack.packs import Jane
wisechat = Jane.user("Author an alliterative poem about good snacks to eat with coffee.")
wisechat.engine = "gpt-4"
wisechat.temperature = 0.8
print(wisechat.ask())

## Serious Snacking
Hungry for more? You've come to the right place!

### Using the Chat object

In [ ]:
from chatsnack import Chat
mychat = Chat()
mychat.system("Respond only with the word POPSICLE from now on.")
mychat.user("What is your name?")
mychat.ask()

Tasty, but pretty vanilla. Let's spice things up a bit.

In [ ]:
# You've had a bite of chaining before, and it works fine here, too.
mychat = (
    Chat()
    .system("Respond only with the word POPSICLE from now on.")
    .user("What is your name?")
    .chat("... and do you even LIKE popsicles?")
)
print(f"This seems familiar:\n\t{mychat.last}\n")

# But a chat object can be shortened down to just calling it like a function with a user message as an argument
newchat = mychat("Are you an AI?")
print(f"Okay, maybe that could be handy:\n\t{newchat.last}\n")

# And we can chain those together, too, but this is just showing off.
popsiclechat = (
    newchat
    ("What is your occupation?")
    ("What is your favorite color?")
    ("Is this expensive to spam OpenAI with?")
    ("What is your favorite food?")
)
print(f"Okay, okay, we get the point:\n\t{popsiclechat.last}\n")
print("(at least the favorite food question made sense)")

Chaining is useful when building a multi-shot prompt quickly, or in cases where chain-of-thought prompting can provide a better response.

In [ ]:
from chatsnack import Chat
popcorn = (
    Chat("Respond with the certainty and creativity of a professional writer.")
    ("Explain 3 rules to writing a clever poem that amazes your friends.")
    ("Using those tips, write a scrumptious poem about popcorn.")
)
print(popcorn.last)

### Let's Chat Inside 
#### Just JSON
If you're familiar with the OpenAI ChatGPT API, it uses a JSON list of messages. You can get the JSON if you want.

In [ ]:
jmessages = popsiclechat.json # POPSICLE time

import json
line_list = json.dumps(json.loads(jmessages), indent=2).split("\n")
for line in line_list:
    print(line)

#### Yummy YAML
But in `chatsnack`, we prefer YAML. It's easy to read and write, and convenient to re-use. Every chat can be a template for later use!

In [ ]:
# Every chat is actually yaml-backed, we can save/load/edit
print(popsiclechat.yaml)

Isn't that nice? Here's what it might look with a default 'chat' instance.

In [ ]:
trychat = Chat()
trychat.system("Respond only with the word PRETZELS from now on.")
print(trychat.yaml)

We can edit that YAML file in a text editor. For building your own library of prompts/chats, this is so much more convenient (and often preferable) to hard-coding all your string prompts in your code.

In [ ]:
trychat.save()

# add another line to the YAML
with trychat.datafile.path.open("a") as f:
    f.write("  - user: What is your name?\n")
print(trychat.datafile.path.read_text())
trychat.load()

# now we can just ask the chat and it uses the new message we added
print(trychat.ask())

We can give a Chat object a name, and it will be stored in the default ./datafiles/chatsnack directory for easy-reuse.

In [ ]:
addict = Chat(name="SnackAddict").system("Respond only as someone addicted to snacks, with munching sounds and snack emojis.")
addict.save()

# we can load that chat back in by name
nowchat = Chat(name="SnackAddict")
nowchat.load()
print(nowchat.ask("How do clouds form?"))


#### Tasty Text (Fillings)
We also have a Text object that's later going to be handy as fillings for your chats. Let's see how it works.

In [ ]:
from chatsnack import Text
mytext = Text(name="SnackExplosion", content="Respond only in explosions of snack emojis and happy faces.")
mytext.save()
# content and file contents are the same
print(mytext.content)
print(mytext.datafile.path.read_text()) 

We can setup Chat objects to pull in Text objects and use them in our chats.

In [ ]:
explosions = Chat(name="SnackSnackExplosions").system("{text.SnackExplosion}")
explosions.ask("What is your name?")

You can even nest these and go deeper...

In [ ]:
anothertext = Text(name="SnackExplosion2", content="{text.SnackExplosion} End every response with 3 more popcorn emojis.")
anothertext.save()

okayokay = Chat(name="SnackSnackExplosions").system("{text.SnackExplosion2}")
print(okayokay.ask("What is your name?"))
print(okayokay.yaml)

#### Nested Chats - Include and Fillings
So we can chain Chats like champs, and we can tuck Texts into templates. Now we're going to chuck Chats into Chats.

There are two ways to do this-- `include` messages and via the `{chat.___}` filling expander.

##### Include Messages
You can add an "include" message to a chat, and it will pull in the messages from another chat. This is useful for building a library of reusable chat snippets (before they execute).

In [ ]:
basechat = Chat(name="ExampleIncludedChat").system("Respond only with the word CARROTSTICKS from now on.")
basechat.save()

anotherchat = Chat().include("ExampleIncludedChat")
print("\nWithout include expansion:\n" + anotherchat.json_unexpanded)
print("\nExpanded:\n" + anotherchat.json)

basechat.user("Another question?")
basechat.save()
print("\nExpanded (showing updates):\n" + anotherchat.json)

print("\nAs YAML:\n" + anotherchat.yaml)

##### Snack Fillings
In `chatsnack`, we call prompt expansion plugins `fillings` and they register (TODO) via the `chatsnack.fillings` module.

There are three types of fillings as of the initial release:
* Text (as seen above)
* Chat 
* dict

The `{chat.___}` *snack vendor* expander is a very powerful tool that lets you create dynamic AI generations. These chat expansions are run in parallel, and the results are combined into a single prompt once they're ready.

In [ ]:
from chatsnack import Chat

# save a chat to nest
snacknames = Chat("FiveSnackNames").system("Respond with high creativity and confidence.").user("Provide 5 random snacks.")
snacknames.save()

# save a second chat to nest
snackdunk = Chat("SnackDunk").system("Respond with high creativity and confidence.").user("Provide 3 dips or drinks are great for snack dipping.")
snackdunk.save()

# build the chat that uses the two above
snackfull = Chat().system("Respond with high confidence.")
snackfull.user("""Choose 1 snack from this list:
{chat.FiveSnackNames}

Choose 1 dunking liquid from this list:
{chat.SnackDunk}

Recommend the best single snack and dip combo above.""")

print("\nBefore snack fillings:\n\n" + snackfull.yaml)
snackout = snackfull.chat()   # save to a different one (rather than overwrite)
print("\nAfter snack fillings and the response:\n\n" + snackout.yaml)

Fillings are also supported with simple keyword replacement, which is probably the most important use case.

In [ ]:
from chatsnack import Chat
healthchat = Chat("Respond only with BOOL: TRUE/FALSE based on your snack expertise.")
healthchat.user("Is {snack_name} a healthy snack?")
healthchat.asst("BOOL: ")

def is_healthy_snack(snack):
    return "true" in healthchat.ask(snack_name=snack).lower()

print("apple == healthy is", is_healthy_snack("apple"))
print("candy == healthy is", is_healthy_snack("candy"))

In [ ]:
from chatsnack import Chat
caloric = Chat("Respond only with {{\"calories\": N}}\" where N is the integer calories, average based on dietician snack expertise for a single portion. Respond only in this format.")
caloric.temperature = 0.0    # deterministic
caloric.user("apple").asst('{{"calories": 52}}')  # 1-shot example
caloric.user("{snack_name}").asst('{{"calories": ')  # make it complete the format

def get_calories(snack):
    return int(caloric.ask(snack_name=snack).split('}')[0])

snacklist = ["apple", "popcorn", "slimjim", "potato salad", "egg"]
for snack in snacklist:
    print(f"{snack} = {get_calories(snack)}")